# Regncon Puljepåmelding

In [ ]:
pip install names

In [1]:
import pandas as pd
from datetime import datetime

In [3]:
#data = pd.read_csv('sjema.csv')
#data = data.drop(data.columns[-2:], axis=1) # Fjerner de to irrelevante kolonnene.

In [3]:
# Colors for text
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [4]:
class deltager():
  def __init__(self, paameldingsdato, email, navn, p1v1,p1v2,p2v1,p2v2,p3v1,p3v2,p4v1,p4v2):
    self.paameldingsdato = paameldingsdato# datetime.strptime(paameldingsdato, '%d/%m/%Y %H:%M:%S') # Husk: Mandag < Tirsdag
    self.email = email
    self.navn = navn
    self.pulje1_valg = [p1v1,p1v2]
    self.pulje2_valg = [p2v1,p2v2]
    self.pulje3_valg = [p3v1,p3v2]
    self.pulje4_valg = [p4v1,p4v2]

    self.pulje1_invilget = None
    self.pulje2_invilget = None
    self.pulje3_invilget = None
    self.pulje4_invilget = None
    
  def get_accepted_activities(self):
    return [self.pulje1_invilget, self.pulje2_invilget, self.pulje3_invilget, self.pulje4_invilget]

  def __str__(self):
    return f'navn: {bcolors.OKGREEN + self.navn + bcolors.ENDC}'#, påmeldingsdato: {self.paameldingsdato}'

  def __eq__(self, other): # Both dates are the same :/
    return self.paameldingsdato == other.paameldingsdato
  
  def __lt__(self, other): # Lower priority if paamelding was later
    return self.paameldingsdato > other.paameldingsdato
      
  def __gt__(self, other): # Higher priority if paamelding was earlier
    return self.paameldingsdato < other.paameldingsdato

In [5]:
class aktivitet():
  def __init__(self, navn, n_plasser):
    self.navn = navn
    self.n_plasser = n_plasser
    self.n_plasser_fylt = 0
    self.deltagere = [None] * n_plasser

  def __str__(self):
    return self.navn

  def isFull(self):
    return self.n_plasser_fylt == self.n_plasser

  def addPerson(self, person):
    if not self.isFull():
      self.deltagere[self.n_plasser_fylt] = person
      self.n_plasser_fylt += 1
      return True
    else:
      return False

  def print_deltagerliste(self):
    print(bcolors.OKBLUE + self.__str__() + bcolors.ENDC)
    if self.n_plasser_fylt == 0:
      print(bcolors.FAIL + 'Ingen påmeldt' + bcolors.ENDC)
    else:
      for i in range(self.n_plasser_fylt):
        print(self.deltagere[i])
    if not self.isFull():
      print(bcolors.WARNING + f'Plasser igjen: {self.n_plasser-self.n_plasser_fylt}' + bcolors.ENDC)

In [6]:
class pulje_alg():
  def __init__(self, deltagere, aktiviteter_pulje_1, aktiviteter_pulje_2, aktiviteter_pulje_3, aktiviteter_pulje_4):
    deltagere.sort(reverse=True)
    self.deltagere = deltagere
    self.aktiviteter_1 = aktiviteter_pulje_1
    self.aktiviteter_2 = aktiviteter_pulje_2
    self.aktiviteter_3 = aktiviteter_pulje_3
    self.aktiviteter_4 = aktiviteter_pulje_4

    self.pulje1_homeless = []
    self.pulje2_homeless = []
    self.pulje3_homeless = []
    self.pulje4_homeless = []

    # Turn activity-string into activity object.
    for d in self.deltagere:
      d.pulje1_valg = [self.aktivitetnavn_til_obj(valg, 1) for valg in d.pulje1_valg]
      d.pulje2_valg = [self.aktivitetnavn_til_obj(valg, 2) for valg in d.pulje2_valg]
      d.pulje3_valg = [self.aktivitetnavn_til_obj(valg, 3) for valg in d.pulje3_valg]
      d.pulje4_valg = [self.aktivitetnavn_til_obj(valg, 4) for valg in d.pulje4_valg]

  # Turn activity name into activity object
  def aktivitetnavn_til_obj(self, navn, puljeNr):
    if puljeNr == 1:
      for a in self.aktiviteter_1:
        if navn == a.navn:
          return a
    if puljeNr == 2:
      for a in self.aktiviteter_2:
        if navn == a.navn:
          return a
    if puljeNr == 3:
      for a in self.aktiviteter_3:
        if navn == a.navn:
          return a
    if puljeNr == 4:
      for a in self.aktiviteter_4:
        if navn == a.navn:
          return a
    print(f'Fant ikke aktiviteten med navn: {navn}')
    return None

  def run(self):
    # Pulje 1
    for d in self.deltagere:
      if not d.pulje1_valg[0].isFull():     # Deltager fikk førstevalg
        d.pulje1_valg[0].addPerson(d)
        d.pulje1_invilget = d.pulje1_valg[0] 
      elif not d.pulje1_valg[1].isFull():   # Deltager fikk andrevalg
        d.pulje1_valg[1].addPerson(d)
        d.pulje1_invilget = d.pulje1_valg[1] 
      else:                                 # Deltager er Homeless
        self.pulje1_homeless.append(d)
        d.pulje1_invilget = 'Homeless' 

    # TODO: Sorter self.deltagere slik at de som er 'homeless1' får førstepri I .
    self.pulje1_homeless.sort() # Sorts from latest to earliest (makes sense in the for loop under.)
    deltagere_for_pulje_2 = [d for d in self.deltagere if d not in self.pulje1_homeless]
    for h in self.pulje1_homeless:
      deltagere_for_pulje_2.insert(0,h)

    # Pulje 2
    for d in deltagere_for_pulje_2:
      if not d.pulje2_valg[0].isFull():
        d.pulje2_valg[0].addPerson(d)
        d.pulje2_invilget = d.pulje2_valg[0] 
      elif not d.pulje2_valg[1].isFull():
        d.pulje2_valg[1].addPerson(d)
        d.pulje2_invilget = d.pulje2_valg[1] 
      else:
        self.pulje2_homeless.append(d)
        d.pulje2_invilget = 'Homeless' 

    # TODO: Sorter self.deltagere slik at de som er 'homeless2' får førstepri.  
    self.pulje2_homeless.sort() # Sorts from latest to earliest (makes sense in the for loop under.)
    deltagere_for_pulje_3 = [d for d in self.deltagere if d not in self.pulje2_homeless]
    for h in self.pulje2_homeless:
      deltagere_for_pulje_3.insert(0,h)


    # Pulje 3
    for d in deltagere_for_pulje_3:
      if not d.pulje3_valg[0].isFull():
        d.pulje3_valg[0].addPerson(d)
        d.pulje3_invilget = d.pulje3_valg[0] 
      elif not d.pulje3_valg[1].isFull():
        d.pulje3_valg[1].addPerson(d)
        d.pulje3_invilget = d.pulje3_valg[1] 
      else:
        self.pulje3_homeless.append(d)
        d.pulje3_invilget = 'Homeless' 

    # TODO: Sorter self.deltagere slik at de som er 'homeless3' får førstepri.
    self.pulje3_homeless.sort() # Sorts from latest to earliest (makes sense in the for loop under.)
    deltagere_for_pulje_4 = [d for d in self.deltagere if d not in self.pulje3_homeless]
    for h in self.pulje3_homeless:
      deltagere_for_pulje_4.insert(0,h)

    # Pulje 4
    for d in deltagere_for_pulje_4:
      if not d.pulje4_valg[0].isFull():
        d.pulje4_valg[0].addPerson(d)
        d.pulje4_invilget = d.pulje4_valg[0] 
      elif not d.pulje4_valg[1].isFull():
        d.pulje4_valg[1].addPerson(d)
        d.pulje4_invilget = d.pulje4_valg[1] 
      else:
        self.pulje4_homeless.append(d)  
        d.pulje4_invilget = 'Homeless'

    return 'Algorithm success B) *Explosion in background*'

  def print_results(self):
    print(bcolors.HEADER + 'pulje 1' + bcolors.ENDC)
    for a in alg.aktiviteter_1:
      a.print_deltagerliste()
    print()
    if len(self.pulje1_homeless) != 0:
      print(bcolors.WARNING + 'The homeless ' + bcolors.ENDC)
      for h in self.pulje1_homeless:
        print(h)

    print()
    print(bcolors.HEADER + 'pulje 2' + bcolors.ENDC)
    for a in alg.aktiviteter_2:
      a.print_deltagerliste()
    print()
    if len(self.pulje2_homeless) != 0:
      print(bcolors.WARNING + 'The homeless ' + bcolors.ENDC)
      for h in self.pulje2_homeless:
        print(h)

    print()
    print(bcolors.HEADER + 'pulje 3' + bcolors.ENDC)
    for a in alg.aktiviteter_3:
      a.print_deltagerliste()
    print()
    if len(self.pulje3_homeless) != 0:
      print(bcolors.WARNING + 'The homeless ' + bcolors.ENDC)
      for h in self.pulje3_homeless:
        print(h)

    print()
    print(bcolors.HEADER + 'pulje 4' + bcolors.ENDC)
    for a in alg.aktiviteter_4:
      a.print_deltagerliste()
    print()
    if len(self.pulje4_homeless) != 0:
      print(bcolors.WARNING + 'The homeless ' + bcolors.ENDC)
      for h in self.pulje4_homeless:
        print(h)

  def get_puljer(self):

    puljer = pd.DataFrame(columns=['Pulje1', 'Pulje2', 'Pulje3', 'Pulje4'], index=[d.navn for d in self.deltagere])

    for d in self.deltagere:
      puljer.loc[d.navn] = d.get_accepted_activities()
    
    print(puljer)

    return puljer

In [ ]:
persons = list()
#for i in range(len(data)): # Går gjennom alle deltagere
#  p = deltager(*data.iloc[i]) # Using spread to insert all parameters
#  persons.append(p)

# SLETT DETTE. ONLY FOR MORE TEST DATA
#for i in range(len(data)): # Går gjennom alle deltagere
#  p = deltager(*data.iloc[i]) # Using spread to insert all parameters
#  p.paameldingsdato = datetime.strptime('09/09/2021 20:33:00', '%d/%m/%Y %H:%M:%S') # Husk: Mandag < Tirsdag
#  p.navn = 'Tidlige Tron'
#  persons.append(p)

import names
for i in range(10):
  p = deltager(datetime.now(), 'mail@gmai.com', names.get_full_name(), 'dnd','ludo','dnd','monopol','ludo','dnd','dnd','dnd2')
  persons.append(p)

In [ ]:
aktiviteter_pulje_1 = [
               aktivitet(navn='dnd', n_plasser=4),
               aktivitet(navn='ludo', n_plasser=4),
               aktivitet(navn='vri-8', n_plasser=4)
]
aktiviteter_pulje_2 = [
               aktivitet(navn='dnd', n_plasser=4),
               aktivitet(navn='monopol', n_plasser=6),
               aktivitet(navn='vri-8', n_plasser=4)
]
aktiviteter_pulje_3 = [
               aktivitet(navn='dnd', n_plasser=4),
               aktivitet(navn='ludo', n_plasser=4),
               aktivitet(navn='vri-8', n_plasser=4)
]
aktiviteter_pulje_4 = [
               aktivitet(navn='dnd', n_plasser=4),
               aktivitet(navn='dnd2', n_plasser=4),
               aktivitet(navn='vri-8', n_plasser=4)
]


In [ ]:
for p in persons:
  print(p)

navn: Bernard Shapiro
navn: Linda Campbell
navn: Margaret Walther
navn: Earnest Coles
navn: David Garza
navn: Wanda Monsen
navn: Mary Cummings
navn: James Palomaki
navn: David Meadows
navn: Mary Joyner


In [ ]:
alg = pulje_alg(persons, aktiviteter_pulje_1, aktiviteter_pulje_2, aktiviteter_pulje_3, aktiviteter_pulje_4)
alg.run()
alg.print_results()

pulje 1
dnd
navn: Bernard Shapiro
navn: Linda Campbell
navn: Margaret Walther
navn: Earnest Coles
ludo
navn: David Garza
navn: Wanda Monsen
navn: Mary Cummings
navn: James Palomaki
vri-8
Ingen påmeldt
Plasser igjen: 4

The homeless 
navn: Mary Joyner
navn: David Meadows

pulje 2
dnd
navn: David Meadows
navn: Mary Joyner
navn: Bernard Shapiro
navn: Linda Campbell
monopol
navn: Margaret Walther
navn: Earnest Coles
navn: David Garza
navn: Wanda Monsen
navn: Mary Cummings
navn: James Palomaki
vri-8
Ingen påmeldt
Plasser igjen: 4


pulje 3
dnd
navn: David Garza
navn: Wanda Monsen
navn: Mary Cummings
navn: James Palomaki
ludo
navn: Bernard Shapiro
navn: Linda Campbell
navn: Margaret Walther
navn: Earnest Coles
vri-8
Ingen påmeldt
Plasser igjen: 4

The homeless 
navn: Mary Joyner
navn: David Meadows

pulje 4
dnd
navn: David Meadows
navn: Mary Joyner
navn: Bernard Shapiro
navn: Linda Campbell
dnd2
navn: Margaret Walther
navn: Earnest Coles
navn: David Garza
navn: Wanda Monsen
vri-8
Ingen påmel

In [ ]:
deltagere = []
for d in alg.deltagere:
  deltagere.append(d.navn)

puljer = pd.DataFrame(columns=['Pulje1', 'Pulje2', 'Pulje3', 'Pulje4'], index=deltagere)

for d in alg.deltagere:
  puljer.loc[d.navn] = d.get_accepted_activities()

puljer

,Pulje1,Pulje2,Pulje3,Pulje4
Bernard Shapiro,dnd,dnd,ludo,dnd
Linda Campbell,dnd,dnd,ludo,dnd
Margaret Walther,dnd,monopol,ludo,dnd2
Earnest Coles,dnd,monopol,ludo,dnd2
David Garza,ludo,monopol,dnd,dnd2
Wanda Monsen,ludo,monopol,dnd,dnd2
Mary Cummings,ludo,monopol,dnd,Homeless
James Palomaki,ludo,monopol,dnd,Homeless
David Meadows,Homeless,dnd,Homeless,dnd
Mary Joyner,Homeless,dnd,Homeless,dnd


In [ ]:
puljer.to_excel('./puljefordeling.xlsx')

In [7]:
def create_pulje(deltagere, aktiviteter):
  navn = []
  for d in deltagere:
    navn.append(d.navn)
  
  alg = pulje_alg(deltagere, aktiviteter[:2], aktiviteter[2:4], aktiviteter[4:6], aktiviteter[6:8])
  print(alg.get_puljer())
  return alg.get_puljer()#pd.DataFrame({'Navn':navn, 'Pulje1':aktiviteter}).set_index('Navn', drop=True)

In [8]:
def should_return_valid_puljefordeling():
  # Given
  expected = pd.DataFrame(index=['Spiller 1', 'Spiller 2', 'Spiller 3'],data={
            'Pulje1':['Spill 1', 'Spill 1', 'Spill 2'],
            'Pulje2':['Spill 3', 'Spill 4', 'Spill 3'],
            'Pulje3':['Spill 5', 'Spill 5', 'Spill 6'],
            'Pulje4':['Spill 7', 'Spill 8', 'Spill 7'],
            })
  deltagere = []
  for i in range(3):
    p = deltager(datetime.now(), 'mail@gmai.com', f'Spiller {i+1}', 'Spill 1','Spill 2','Spill 3','Spill 4','Spill 5','Spill 6','Spill 7','Spill 8')
    deltagere.append(p)

  aktiviteter = []
  for i in range(8):
    aktiviteter.append(aktivitet(f'Spill {i+1}', 2))

  # When
  data_to_test = create_pulje(deltagere, aktiviteter)

  # Then
  return pd.testing.assert_frame_equal(data_to_test, expected)
  
should_return_valid_puljefordeling()

          Pulje1 Pulje2 Pulje3 Pulje4
Spiller 1   None   None   None   None
Spiller 2   None   None   None   None
Spiller 3   None   None   None   None
          Pulje1 Pulje2 Pulje3 Pulje4
Spiller 1   None   None   None   None
Spiller 2   None   None   None   None
Spiller 3   None   None   None   None
          Pulje1 Pulje2 Pulje3 Pulje4
Spiller 1   None   None   None   None
Spiller 2   None   None   None   None
Spiller 3   None   None   None   None


AssertionError: DataFrame.iloc[:, 0] (column name="Pulje1") are different

DataFrame.iloc[:, 0] (column name="Pulje1") values are different (100.0 %)
[index]: [Spiller 1, Spiller 2, Spiller 3]
[left]:  [None, None, None]
[right]: [Spill 1, Spill 1, Spill 2]